<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Interface

## Código

In [239]:
# ===================================================================
# Bloco 1: Interface do Usuário
# ===================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output

class LogColors: INFO, SUCCESS, ERROR, WARNING, RESET = ('\033[94m', '\033[92m', '\033[91m', '\033[93m', '\033[0m')

# --- Widgets ---
code_area = widgets.Textarea(value="", placeholder="Escreva seu código assembly aqui...", description="Code:", layout=widgets.Layout(width="100%", height="200px"))
output_area = widgets.Output()
part_selector = widgets.RadioButtons(options=[('Ciclo Único', 'p1_2'), ('Pipeline (Completo)', 'p_extra')], description='Simular:')
hazard_mode_selector = widgets.RadioButtons(options=[('Sem tratamento de hazards', 'none'), ('Stall em hazards', 'stall'), ('Forwarding', 'forwarding')], value='forwarding', description='Hazards:')
cache_selector = widgets.RadioButtons(options=[('Com Cache', 'enabled'), ('Sem Cache', 'disabled')], value='enabled', description='Memória:')
cache_policy_selector = widgets.RadioButtons(options=[('LRU', 'lru'), ('Random', 'random')], value='lru', description='Política Cache:')
cache_penalty_input = widgets.IntText(value=10, description="Penalidade Miss:")

# --- Botões de Ação ---
load_button = widgets.Button(description="Load/Reset", button_style="primary")
# DOIS botões "Próximo Passo", visualmente idênticos
step_single_cycle_button = widgets.Button(description="Próximo Passo ->", button_style="success", icon='step-forward')
step_pipeline_button = widgets.Button(description="Próximo Ciclo ->", button_style="success", icon='step-forward')
run_all_button = widgets.Button(description="Run All", button_style="info", icon='fast-forward')

reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")
cache_button = widgets.Button(description="Show Cache Stats", button_style="warning")
start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

# --- Handlers ---
def load_program_handler(b):
    with output_area: clear_output(wait=True)
    clear_output(wait=True); display(ui)

    if part_selector.value == 'p1_2':
        initialize_single_cycle_simulation()
    else: # Pipeline
        initialize_pipeline_simulation()

def step_single_cycle_handler(b): execute_one_single_cycle_step()
def step_pipeline_handler(b): execute_one_pipeline_cycle()

def run_all_handler(b):
    # Primeiro, carrega/reseta o estado
    load_program_handler(None)
    print(f"\n{LogColors.INFO}--- Executando todas as etapas ---{LogColors.RESET}")
    if part_selector.value == 'p1_2':
        while not execute_one_single_cycle_step(): # Loop até a função retornar "finalizado"
            pass
    else: # Pipeline
        while not execute_one_pipeline_cycle(): # Loop até a função retornar "finalizado"
            pass

def on_part_selection_change(change):
    is_pipeline = (change['new'] == 'p_extra')
    step_single_cycle_button.layout.display = 'inline-block' if not is_pipeline else 'none'
    step_pipeline_button.layout.display = 'inline-block' if is_pipeline else 'none'
    for widget in [hazard_mode_selector, cache_selector, cache_policy_selector, cache_penalty_input, cache_button]:
        widget.layout.display = 'flex' if is_pipeline else 'none'
    if is_pipeline: on_cache_selection_change({'new': cache_selector.value})
    else: cache_policy_selector.layout.display = 'none'; cache_penalty_input.layout.display = 'none'

def on_cache_selection_change(change):
    is_cache_enabled = (change['new'] == 'enabled')
    cache_policy_selector.layout.display = 'flex' if is_cache_enabled else 'none'
    cache_penalty_input.layout.display = 'flex' if is_cache_enabled else 'none'

part_selector.observe(on_part_selection_change, names='value')
cache_selector.observe(on_cache_selection_change, names='value')

# --- Conexões e Layout ---
load_button.on_click(load_program_handler)
step_single_cycle_button.on_click(step_single_cycle_handler)
step_pipeline_button.on_click(step_pipeline_handler)
run_all_button.on_click(run_all_handler)
reg_button.on_click(reg_button_handler); mem_button.on_click(mem_button_handler); cache_button.on_click(cache_button_handler)

regs, mem, cache = None, None, None
top_controls = widgets.HBox([load_button, step_single_cycle_button, step_pipeline_button, run_all_button, reg_button, mem_button, cache_button, start_addr, mem_range])
pipeline_options = widgets.VBox([hazard_mode_selector, cache_selector, cache_policy_selector, cache_penalty_input])
ui = widgets.VBox([part_selector, pipeline_options, code_area, top_controls, output_area])

on_part_selection_change({'new': part_selector.value})


## Execução

In [240]:
display(ui)

Programa carregado com 5 instruções. Pronto para iniciar.

--- Executando todas as etapas ---
Passo 1: ADDI     x1, x0, 1        -> Sucesso!
Passo 2: ADDI     x2, x0, 2        -> Sucesso!
Passo 3: ADDI     x3, x0, 3        -> Sucesso!
Passo 4: ADDI     x4, x0, 4        -> Sucesso!
Passo 5: ADDI     x5, x0, 5        -> Sucesso!


# Componetes Comuns

In [241]:
# ===================================================================
# Bloco 2: Componentes Comuns
# ===================================================================

from IPython.display import display, HTML
import pandas as pd
import struct

# --- Classe de Cores para o Log ---
class LogColors: INFO, SUCCESS, ERROR, WARNING, RESET = ('\033[94m', '\033[92m', '\033[91m', '\033[93m', '\033[0m')

# --- Classes do Simulador ---
class SimpleMemory:
    def __init__(self, size_in_words=64): self.size = size_in_words; self.mem = [0] * size_in_words
    def load_word(self, address):
        if 0 <= address < self.size: return self.mem[address]
        raise IndexError(f"Endereço de memória inválido: {address}")
    def store_word(self, address, value):
        if 0 <= address < self.size: self.mem[address] = value & 0xFFFFFFFF
        else: raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    def __init__(self): self.regs = [0] * 32
    def read(self, reg_num):
        if 0 <= reg_num < 32: return self.regs[reg_num]
        raise IndexError(f"Número de registrador inválido: x{reg_num}")
    def write(self, reg_num, value):
        if reg_num == 0: return
        if 0 < reg_num < 32: self.regs[reg_num] = value & 0xFFFFFFFF
        else: raise IndexError(f"Número de registrador inválido: x{reg_num}")

# --- Funções Auxiliares ---
def float_to_int_bits(f): return struct.unpack('I', struct.pack('f', float(f)))[0]
def int_bits_to_float(i):
    try: return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except: return float('nan')
def to_signed32(value):
    value &= 0xFFFFFFFF
    return value - (1 << 32) if value & (1 << 31) else value
def format_instruction(instr):
    if not instr: return ""
    op = instr['op']
    if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, x{instr['rs2']}"
    if op == "ADDI": return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, {instr['imm']}"
    if op == "LW": return f"{op:<8} x{instr['rd']}, {instr['addr']}"
    if op == "SW": return f"{op:<8} x{instr['rs1']}, {instr['addr']}"
    return str(instr)

# --- Funções de Exibição (COM CORREÇÃO) ---
def show_registers(regs):
    if not regs:
        with output_area:
            clear_output(wait=True)
            print(f"{LogColors.WARNING}Programa não executado. Clique em 'Run Program' primeiro.{LogColors.RESET}")
        return

    output_widgets = []
    for i in range(0, 32, 8):
        start_reg, end_reg = i, i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        data = {"Reg": [f"x{j}" for j in range(start_reg, end_reg)], "Valor Int": [to_signed32(val) for val in register_values], "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]}
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output()
        with out: display(df)
        output_widgets.append(out)
    with output_area:
        clear_output(wait=True)
        display(widgets.HBox(output_widgets))

def show_memory(mem, start=0, rng=10):
    if not mem:
        with output_area:
            clear_output(wait=True)
            print(f"{LogColors.WARNING}Programa não executado. Clique em 'Run Program' primeiro.{LogColors.RESET}")
        return

    try:
        data = {"Endereço": list(range(start, start + rng)), "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        df = pd.DataFrame(data)
        with output_area: clear_output(wait=True); display(df)
    except IndexError as e:
        with output_area: clear_output(wait=True); print(f"{LogColors.ERROR}Erro ao acessar memória: {e}{LogColors.RESET}")

def show_cache_stats(cache_obj):
    if not cache_obj:
        with output_area:
            clear_output(wait=True)
            print(f"{LogColors.WARNING}Programa não executado. Clique em 'Run Program' primeiro.{LogColors.RESET}")
        return
    hit_rate, miss_rate, hits, misses = cache_obj.get_stats()
    stats_str = (
        f"{LogColors.INFO}--- Estatísticas da Cache ---\n{LogColors.RESET}"
        f"Acessos Totais: {hits + misses}\n"
        f"Hits: {LogColors.SUCCESS}{hits}{LogColors.RESET}\n"
        f"Misses: {LogColors.ERROR}{misses}{LogColors.RESET}\n"
        f"Taxa de Acertos (Hit Rate): {LogColors.SUCCESS}{hit_rate:.2f}%{LogColors.RESET}\n"
        f"Taxa de Falhas (Miss Rate): {LogColors.ERROR}{miss_rate:.2f}%{LogColors.RESET}"
    )
    with output_area:
        clear_output(wait=True)
        print(stats_str)

# --- Função para gerar o botão de "Copiar" ---
def create_copy_button(code_text, button_id):
    area_id, btn_id = f"code-area-{button_id}", f"copy-button-{button_id}"
    html_content = f"""
    <div style="position: relative; border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin-bottom: 15px; background-color: #2e2e2e; color: #f0f0f0;">
        <pre style="margin: 0; white-space: pre-wrap; word-wrap: break-word;"><code>{code_text}</code></pre>
        <button id="{btn_id}" onclick="copyToClipboard('{area_id}', '{btn_id}')" style="position: absolute; top: 10px; right: 10px; padding: 5px 10px; border: 1px solid #555; border-radius: 5px; cursor: pointer; background-color: #444; color: #fff;">
            Copiar
        </button>
        <textarea id="{area_id}" style="position: absolute; left: -9999px; top: 0;">{code_text}</textarea>
    </div>
    <script>
    if (typeof copyToClipboard === 'undefined') {{
        function copyToClipboard(elementId, buttonId) {{
            const codeElement = document.getElementById(elementId);
            codeElement.select();
            codeElement.setSelectionRange(0, 99999);
            navigator.clipboard.writeText(codeElement.value).then(() => {{
                const button = document.getElementById(buttonId);
                button.innerText = 'Copiado!';
                button.style.backgroundColor = '#3a8a3a';
                setTimeout(() => {{ button.innerText = 'Copiar'; button.style.backgroundColor = '#444'; }}, 2000);
            }}).catch(err => console.error('Falha ao copiar texto: ', err));
        }}
    }}
    </script>
    """
    return HTML(html_content)

# Parte 2

In [242]:
# ===================================================================
# Bloco 3: Lógica da Parte 2 (Simulador de Ciclo Único)
# ===================================================================
def execute_instruction_single_cycle(instr, regs, mem):
    op = instr["op"]
    if op in ["ADD", "SUB", "MUL"]:
        rs1_val, rs2_val = to_signed32(regs.read(instr["rs1"])), to_signed32(regs.read(instr["rs2"]))
        if op == "ADD": result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        else: result = rs1_val * rs2_val
        regs.write(instr["rd"], result)
    elif op == "LW": regs.write(instr["rd"], mem.load_word(instr["addr"]))
    elif op == "SW": mem.store_word(instr["addr"], regs.read(instr["rs1"]))
    else: raise ValueError(f"Instrução '{op}' não pertence ao conjunto base da Parte 2.")

def parse_program_single_cycle(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else: raise ValueError(f"Instrução '{op}' não reconhecida pelo parser base.")
        program.append(instr)
    return program

def run_program_single_cycle(asm_code, regs, mem):
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    for cycle, instr in enumerate(program, 1):
        formatted_instr = format_instruction(instr)
        try:
            execute_instruction_single_cycle(instr, regs, mem)
            print(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.SUCCESS}Sucesso!{LogColors.RESET}")
        except (ValueError, IndexError, OverflowError) as e:
            print(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.ERROR}ERRO: {e}{LogColors.RESET}")

print(f"{LogColors.SUCCESS}Funções base de simulação (Parte 2) carregadas.{LogColors.RESET}")

Funções base de simulação (Parte 2) carregadas.


# Parte 1


In [243]:
# ===================================================================
# Bloco 4: Lógica da Parte 1 (Extensões para o Simulador)
#
# Este bloco REDEFINE as funções de parse e execute do Bloco 3,
# adicionando as funcionalidades da Parte 1 (DIV, ADDI, Ponto
# Flutuante, Overflow).
# ===================================================================

def execute_instruction_single_cycle(instr, regs, mem):
    """Versão completa: Executa instruções das Partes 1 e 2."""
    op = instr["op"]
    INT32_MIN, INT32_MAX = -2**31, 2**31 - 1

    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]))
        rs2_val = instr["imm"] if op == "ADDI" else to_signed32(regs.read(instr["rs2"]))
        op_symbol = {'ADD': '+', 'ADDI': '+', 'SUB': '-', 'MUL': '*', 'DIV': '/'}[op]

        if op in ["ADD", "ADDI"]: result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val
        elif op == "DIV":
            if rs2_val == 0: raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1: raise OverflowError(f"Overflow na divisão")
            result = int(rs1_val / rs2_val)

        if not (INT32_MIN <= result <= INT32_MAX) and op != "DIV":
            raise OverflowError(f"Overflow na operação: {rs1_val} {op_symbol} {rs2_val}")

        regs.write(instr["rd"], result)

    elif op in ["ADD.S", "MUL.S"]:
        float1, float2 = int_bits_to_float(regs.read(instr["rs1"])), int_bits_to_float(regs.read(instr["rs2"]))
        result_float = (float1 + float2) if op == "ADD.S" else (float1 * float2)
        regs.write(instr["rd"], float_to_int_bits(result_float))

    elif op == "LW":
        regs.write(instr["rd"], mem.load_word(instr["addr"]))

    elif op == "SW":
        mem.store_word(instr["addr"], regs.read(instr["rs1"]))

    else:
        raise ValueError(f"Instrução desconhecida {op}")

def parse_program_single_cycle(asm_code):
    """Versão completa: Reconhece instruções das Partes 1 e 2."""
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]:
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI":
            instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW":
            instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW":
            instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else:
            raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

print(f"{LogColors.SUCCESS}Funções de simulação da Parte 1 e 2 (versão completa para Step-by-Step) carregadas.{LogColors.RESET}")

Funções de simulação da Parte 1 e 2 (versão completa para Step-by-Step) carregadas.


# Parte 3

In [244]:
# ===================================================================
# Bloco 5: Lógica da Parte 3 (Simulador de Pipeline - Base)
# ===================================================================
def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='none'): # Adicionado 'cache' para consistência
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    pc, cycle = 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history = []
    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        pipeline_stages['MEM_WB'] = prev_stages['EX']
        pipeline_stages['EX'] = prev_stages['ID']
        pipeline_stages['ID'] = prev_stages['IF']
        if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
        else: pipeline_stages['IF'] = None
        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB']) })
        if cycle > len(program) + 10: print(f"{LogColors.WARNING}Simulação parada: limite de ciclos.{LogColors.RESET}"); break
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area: display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções base de simulação de Pipeline (Parte 3) carregadas.{LogColors.RESET}")

Funções base de simulação de Pipeline (Parte 3) carregadas.


# Parte 4

In [245]:
# ===================================================================
# Bloco 6: Lógica da Parte 4 (Pipeline com Hazards)
# ===================================================================
def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding'): # Adicionado 'cache'
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    pc, cycle, stalls = 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []
    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        stall_this_cycle = False
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            if 'result' in instr_wb: regs.write(instr_wb['rd'], instr_wb['result'])
            elif instr_wb['op'] == 'SW': mem.store_word(instr_wb['addr'], regs.read(instr_wb['rs1']))
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB']
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem:
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            elif op == "LW": instr_ex['result'] = mem.load_word(instr_ex['addr'])
            pipeline_stages['MEM_WB'] = instr_ex
        else: pipeline_stages['MEM_WB'] = None
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None}
            instr_ex_prev = prev_stages['EX']
            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                if instr_ex_prev['op'] == 'LW':
                    if hazard_mode != 'none': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL OBRIGATÓRIO.{LogColors.RESET}")
                    else: hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
                else:
                    if hazard_mode == 'stall': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.WARNING}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL.{LogColors.RESET}")
                    elif hazard_mode == 'none': hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
        if stall_this_cycle: stalls += 1; pipeline_stages['EX'] = None
        else:
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
            else: pipeline_stages['IF'] = None
        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
        if cycle > (len(program) + stalls) * 2 and cycle > 20: print(f"{LogColors.WARNING}Simulação parada.{LogColors.RESET}"); break
    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Forwarding ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history: df = pd.DataFrame(history).set_index('Ciclo'); display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (bolhas):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções de simulação de Pipeline com Hazards (Parte 4) carregadas.{LogColors.RESET}")

Funções de simulação de Pipeline com Hazards (Parte 4) carregadas.


# Parte 5

In [246]:
# ===================================================================
# Bloco 7: Lógica da Parte 5 (Hierarquia de Memória - Cache)
# ===================================================================
import random

class Cache:
    def __init__(self, num_sets, main_memory, replacement_policy='lru'):
        self.num_sets = num_sets
        self.main_memory = main_memory
        self.replacement_policy = replacement_policy
        self.sets = [[self._create_cache_line(), self._create_cache_line()] for _ in range(num_sets)]
        self.hits, self.misses = 0, 0

    def _create_cache_line(self): return {'valid': 0, 'tag': 0, 'data': 0, 'lru': 0}
    def _get_address_parts(self, address): return address % self.num_sets, address // self.num_sets

    def read(self, address):
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]
        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag:
                self.hits += 1
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i
                return line['data'], 'hit'

        self.misses += 1
        data = self.main_memory.load_word(address)
        if self.replacement_policy == 'lru':
            way_to_replace = next(i for i, line in enumerate(target_set) if line['lru'] != i)
        else: # random
            way_to_replace = random.choice([0, 1])
        target_set[way_to_replace].update({'valid': 1, 'tag': tag, 'data': data})
        target_set[0]['lru'], target_set[1]['lru'] = (1-way_to_replace), way_to_replace
        return data, 'miss'

    def write(self, address, data):
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]
        self.main_memory.store_word(address, data)
        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag:
                self.hits += 1
                line['data'] = data
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i
                return 'hit'
        self.misses += 1
        return 'miss'

    def get_stats(self):
        total = self.hits + self.misses
        if total == 0: return 0.0, 0.0, 0, 0
        hit_rate, miss_rate = (self.hits / total) * 100, (self.misses / total) * 100
        return hit_rate, miss_rate, self.hits, self.misses

print(f"{LogColors.SUCCESS}Componente de Cache com Políticas de Substituição (Parte Extra) carregado.{LogColors.RESET}")

Componente de Cache com Políticas de Substituição (Parte Extra) carregado.


In [247]:
# ===================================================================
# Bloco 8: Integração Final (Pipeline + Cache)
# ===================================================================

def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding', cache_mode='enabled'):
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return

    pc, cycle, stalls = 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []

    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        stall_this_cycle = False

        # ESTÁGIO 4: MEMORY ACCESS & WRITE BACK (MEM_WB)
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            op = instr_wb['op']

            # --- LÓGICA DE ACESSO CONDICIONAL À MEMÓRIA ---
            if op == "LW":
                address = instr_wb['addr']
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    instr_wb['result'] = cache.read(address)
                else: # Acesso direto à memória
                    instr_wb['result'] = mem.load_word(address)
                regs.write(instr_wb['rd'], instr_wb['result'])

            elif op == "SW":
                address = instr_wb['addr']
                data = regs.read(instr_wb['rs1'])
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    cache.write(address, data)
                else: # Acesso direto à memória
                    mem.store_word(address, data)

            elif 'result' in instr_wb: # Instruções Aritméticas
                regs.write(instr_wb['rd'], instr_wb['result'])

        # ESTÁGIO 3: EXECUTE (EX)
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB']
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            pipeline_stages['MEM_WB'] = instr_ex
        else: pipeline_stages['MEM_WB'] = None

        # ESTÁGIO 2: INSTRUCTION DECODE (ID)
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None}
            instr_ex_prev = prev_stages['EX']
            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                if instr_ex_prev['op'] == 'LW':
                    if hazard_mode != 'none': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL OBRIGATÓRIO.{LogColors.RESET}")
                    else: hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
                else:
                    if hazard_mode == 'stall': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.WARNING}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL.{LogColors.RESET}")
                    elif hazard_mode == 'none': hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")

        if stall_this_cycle: stalls += 1; pipeline_stages['EX'] = None
        else:
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
            else: pipeline_stages['IF'] = None

        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
        if cycle > (len(program) + stalls) * 2 and cycle > 30: print(f"{LogColors.WARNING}Simulação parada.{LogColors.RESET}"); break

    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Forwarding ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area:
            display(df.fillna(""))

    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (bolhas):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções de simulação de Pipeline com Cache (Parte 5 - FINAL) carregadas.{LogColors.RESET}")

Funções de simulação de Pipeline com Cache (Parte 5 - FINAL) carregadas.


# Parte Extra

In [248]:
# ===================================================================
# Bloco 9: Parte Extra (Lógica de Execução Final e Step-by-Step)
#
# Este bloco contém as versões finais e interativas das funções de
# simulação, permitindo a execução passo a passo tanto para o
# Ciclo Único quanto para o Pipeline.
# ===================================================================

# --- Parser Final (com labels e desvios) ---
def parse_program_final(asm_code):
    lines = [line.strip().split('#')[0].strip() for line in asm_code.strip().splitlines() if line.strip() and not line.strip().startswith('#')]
    labels, program_lines = {}, []
    current_addr = 0
    for line in lines:
        if ':' in line:
            label, rest = line.split(':', 1)
            labels[label.strip()] = current_addr
            if rest.strip(): program_lines.append(rest.strip())
            else: continue
        else:
            program_lines.append(line)
        current_addr += 1

    program = []
    for line in program_lines:
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI": instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "BEQ": instr = {"op": op, "rs1": int(parts[1][1:]), "rs2": int(parts[2][1:]), "target": labels[parts[3]]}
        elif op == "J": instr = {"op": op, "target": labels[parts[1]]}
        else: raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

# --- Funções de Simulação para Ciclo Único ---
def initialize_single_cycle_simulation():
    global sc_program, sc_pc, regs, mem
    regs = RegisterFile(); mem = SimpleMemory(64)
    try:
        sc_program = parse_program_single_cycle(code_area.value)
        sc_pc = 0
        print(f"{LogColors.SUCCESS}Programa carregado com {len(sc_program)} instruções. Pronto para iniciar.{LogColors.RESET}")
        show_registers(regs)
    except (ValueError, KeyError) as e:
        print(f"{LogColors.ERROR}ERRO DE SINTAXE AO CARREGAR: {e}{LogColors.RESET}"); sc_program, sc_pc = [], 0

def execute_one_single_cycle_step():
    global sc_pc, sc_program, regs, mem
    if not sc_program or sc_pc >= len(sc_program):
        print(f"{LogColors.SUCCESS}Fim do programa.{LogColors.RESET}"); return True
    instr = sc_program[sc_pc]
    formatted_instr = format_instruction(instr)
    try:
        execute_instruction_single_cycle(instr, regs, mem)
        print(f"{LogColors.INFO}Passo {sc_pc+1}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.SUCCESS}Sucesso!{LogColors.RESET}")
    except (ValueError, IndexError, OverflowError) as e:
        print(f"{LogColors.INFO}Passo {sc_pc+1}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.ERROR}ERRO: {e}{LogColors.RESET}")
    sc_pc += 1
    show_registers(regs)
    return sc_pc >= len(sc_program)

# --- Funções de Simulação para Pipeline ---
def initialize_pipeline_simulation():
    global p_program, p_pc, p_cycle, p_stalls, p_mem_stalls, p_pipeline_stages, p_history, p_hazard_logs, regs, mem, cache
    regs = RegisterFile(); mem = SimpleMemory(64)
    cache = Cache(num_sets=8, main_memory=mem, replacement_policy=cache_policy_selector.value)
    try:
        p_program = parse_program_final(code_area.value)
        p_pc, p_cycle, p_stalls, p_mem_stalls = 0, 0, 0, 0
        p_pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
        p_history, p_hazard_logs = [], []
        print(f"{LogColors.SUCCESS}Programa carregado com {len(p_program)} instruções. Pronto para iniciar.{LogColors.RESET}")
        with output_area: clear_output(wait=True)
    except (ValueError, KeyError) as e:
        print(f"{LogColors.ERROR}ERRO DE SINTAXE AO CARREGAR: {e}{LogColors.RESET}"); p_program = []

def execute_one_pipeline_cycle():
    global p_program, p_pc, p_cycle, p_stalls, p_mem_stalls, p_pipeline_stages, p_history, p_hazard_logs, regs, mem, cache
    if p_pc >= len(p_program) and all(s is None for s in p_pipeline_stages.values()):
        finalize_pipeline_simulation(); return True

    p_cycle += 1
    hazard_mode = hazard_mode_selector.value
    cache_mode = cache_selector.value
    miss_penalty = cache_penalty_input.value

    if p_mem_stalls > 0:
        p_mem_stalls -= 1
        p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.WARNING}STALL DE MEMÓRIA (Cache Miss){LogColors.RESET}")
        p_history.append({ 'Ciclo': p_cycle, 'IF': "STALL", 'ID': "STALL", 'EX': "STALL", 'MEM_WB': format_instruction(p_pipeline_stages['MEM_WB'])})
        return False

    prev_stages = p_pipeline_stages.copy()
    stall_this_cycle = False

    # ESTÁGIO 4: MEM_WB
    instr_wb = prev_stages['MEM_WB']
    if instr_wb:
        op = instr_wb['op']
        if op == "LW":
            data, status = cache.read(instr_wb['addr']) if cache_mode == 'enabled' else (mem.load_word(instr_wb['addr']), 'hit')
            if status == 'miss' and miss_penalty > 1: p_mem_stalls = miss_penalty - 1
            regs.write(instr_wb['rd'], data)
        elif op == "SW":
            _, status = cache.write(instr_wb['addr'], regs.read(instr_wb['rs1'])) if cache_mode == 'enabled' else (mem.store_word(instr_wb['addr'], regs.read(instr_wb['rs1'])), 'hit')
            if status == 'miss' and miss_penalty > 1: p_mem_stalls = miss_penalty - 1
        elif 'result' in instr_wb: regs.write(instr_wb['rd'], instr_wb['result'])

    # ESTÁGIO 3: EX
    instr_ex = prev_stages['EX']
    flush_if_id = False
    if instr_ex:
        op = instr_ex['op']
        if op == 'BEQ' and regs.read(instr_ex['rs1']) == regs.read(instr_ex['rs2']):
            p_pc = instr_ex['target']; flush_if_id = True
            p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.ERROR}CONTROL HAZARD (BEQ){LogColors.RESET}. Desvio tomado, flush IF/ID.")
        elif op == 'J':
            p_pc = instr_ex['target']; flush_if_id = True
            p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.ERROR}CONTROL HAZARD (J){LogColors.RESET}. Desvio tomado, flush IF/ID.")
        else:
            val1, val2 = regs.read(instr_ex.get('rs1',0)), regs.read(instr_ex.get('rs2',0)) if 'rs2' in instr_ex else instr_ex.get('imm',0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB']
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.SUCCESS}FWD(MEM->EX){LogColors.RESET} x{fwd_reg} p/ '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.SUCCESS}FWD(MEM->EX){LogColors.RESET} x{fwd_reg} p/ '{format_instruction(instr_ex)}'")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
        p_pipeline_stages['MEM_WB'] = instr_ex
    else: p_pipeline_stages['MEM_WB'] = None

    # ESTÁGIO 2: ID
    instr_id = prev_stages['ID']
    if instr_id:
        src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None}
        instr_ex_prev = prev_stages['EX']
        if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] in src_regs:
            hazard_reg = instr_ex_prev['rd']
            if instr_ex_prev['op'] == 'LW':
                if hazard_mode != 'none': stall_this_cycle = True; p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE{LogColors.RESET} x{hazard_reg}. Ação: STALL.")
                else: p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE{LogColors.RESET} x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
            else:
                if hazard_mode == 'stall': stall_this_cycle = True; p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.WARNING}HAZARD DE DADOS{LogColors.RESET} x{hazard_reg}. Ação: STALL.")
                elif hazard_mode == 'none': p_hazard_logs.append(f"{LogColors.INFO}Ciclo {p_cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE DADOS{LogColors.RESET} x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")

    # AVANÇO DO PIPELINE
    if stall_this_cycle: p_stalls += 1; p_pipeline_stages['EX'] = None
    elif flush_if_id: p_pipeline_stages['EX'] = prev_stages['ID']; p_pipeline_stages['ID'] = p_pipeline_stages['IF'] = None
    else:
        p_pipeline_stages['EX'] = prev_stages['ID']
        p_pipeline_stages['ID'] = prev_stages['IF']
        if p_pc < len(p_program): p_pipeline_stages['IF'] = p_program[p_pc]; p_pc += 1
        else: p_pipeline_stages['IF'] = None

    p_history.append({ 'Ciclo': p_cycle, 'IF': format_instruction(p_pipeline_stages['IF']), 'ID': format_instruction(p_pipeline_stages['ID']), 'EX': f"*{format_instruction(p_pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(p_pipeline_stages['EX']), 'MEM_WB': format_instruction(p_pipeline_stages['MEM_WB'])})
    df = pd.DataFrame(p_history).set_index('Ciclo')
    with output_area: clear_output(wait=True); display(df.fillna(""))

    return False

def finalize_pipeline_simulation():
    global p_program, p_cycle, p_stalls, p_history, p_hazard_logs
    if p_hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Stalls ---{LogColors.RESET}"); [print(log) for log in p_hazard_logs]
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {p_cycle}")
    print(f"{LogColors.INFO}Número de Stalls (Data Hazard):{LogColors.RESET} {p_stalls}")
    print(f"{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(p_program)}")
    speedup = len(p_program) / p_cycle if p_cycle > 0 else 0
    print(f"{LogColors.SUCCESS}Speedup (vs Ciclo Único): {speedup:.2f}x{LogColors.RESET}")
    print(f"\n{LogColors.SUCCESS}Fim do programa.{LogColors.RESET}")

print(f"{LogColors.SUCCESS}Funções de simulação da Parte Extra (com Step-by-Step) carregadas.{LogColors.RESET}")

Funções de simulação da Parte Extra (com Step-by-Step) carregadas.


# Conclusões

## Codigos dos testes

In [249]:
codigo_exp1 = """# --- Teste para as Partes 1 & 2 ---

# Setup de valores
ADDI x1, x0, 100
ADDI x2, x0, 10
ADDI x3, x0, 7

# Teste de operações de inteiros
ADD x4, x1, x2  # x4 = 100 + 10 = 110
SUB x5, x1, x2  # x5 = 100 - 10 = 90
MUL x6, x2, x3  # x6 = 10 * 7 = 70
DIV x7, x1, x2  # x7 = 100 / 10 = 10

# Teste de Overflow (deve falhar e continuar)
ADDI x8, x0, 2147483647
ADD  x9, x8, x8

# Teste de Divisão por Zero (deve falhar e continuar)
DIV x10, x1, x0

# Teste de Ponto Flutuante
# Carrega os bits de 2.5 (1075838976) e 1.5 (1069547520)
ADDI x11, x0, 1075838976
ADDI x12, x0, 1069547520
ADD.S x13, x11, x12 # x13 = 2.5 + 1.5 = 4.0
MUL.S x14, x11, x12 # x14 = 2.5 * 1.5 = 3.75"""

In [250]:
codigo_exp2 = """# --- Teste da Parte 3: Pipeline Básico ---

# Sequência de 5 instruções independentes
ADDI x1, x0, 1
ADDI x2, x0, 2
ADDI x3, x0, 3
ADDI x4, x0, 4
ADDI x5, x0, 5"""

In [251]:
codigo_exp3 = """# --- Teste da Parte 4: Data Hazards ---

# Hazard de Dados (EX -> ID)
ADDI x1, x0, 10
ADDI x2, x0, 20
ADD  x3, x1, x2  # x3 é escrito aqui
SUB  x4, x3, x5  # x3 é lido imediatamente -> HAZARD!

# Hazard de Load-Use (LW -> EX)
ADDI x10, x0, 100
SW   x10, 40     # Armazena 100 no endereço 40
LW   x6, 40      # x6 é escrito com o valor da memória aqui
ADD  x7, x6, x1  # x6 é lido imediatamente -> HAZARD de LW!"""

In [252]:
codigo_exp4 = """# --- Teste da Parte 5: Cache ---
# Cache: 8 conjuntos, 2 vias. Endereços 10, 18 e 26 mapeiam para o conjunto 2.

ADDI x1, x0, 100
ADDI x2, x0, 200

# 1. Miss de Compulsão e Hit Temporal
SW x1, 10   # MISS
LW x3, 10   # HIT

# 2. Miss de Conflito (LRU vs Random pode variar)
SW x2, 18   # MISS no conjunto 2
SW x1, 26   # MISS no conjunto 2 (pode expulsar 10 ou 18)

# 3. Teste da Política de Substituição
LW x4, 10   # Será HIT ou MISS? Depende se o bloco foi expulso."""

In [253]:
codigo_exp5 = """# --- Teste Final: Desvios e Speedup ---
# Um loop simples que soma 10, 5 vezes.
    ADDI x1, x0, 5      # Contador do loop
    ADDI x2, x0, 0      # Acumulador (resultado)
    ADDI x3, x0, 10     # Valor a ser somado

loop:
    ADD  x2, x2, x3     # Acumula o valor
    ADDI x1, x1, -1     # Decrementa o contador
    BEQ  x1, x0, end    # Se contador for zero, vai para 'end'
    J    loop           # Pula de volta para o início do loop

end:
    SW x2, 50           # Salva o resultado final (50) na memória"""

## <strong>Experimento 1: Validação das Partes 1 & 2 (Operações e Ciclo Único)</strong>

<strong>Objetivo:</strong> Validar todas as operações aritméticas de inteiros e ponto flutuante, incluindo a detecção de erros (overflow, divisão por zero), no modo de ciclo único.

<strong>Código Assembly:</strong>


In [254]:
display(create_copy_button(codigo_exp1, button_id=1))

Configurações do Simulador:
- <strong>Simular:</strong> ```Ciclo Único```

Análise dos Resultados Esperados:
- <strong>Log de Execução</strong>: Você deverá ver mensagens de <strong>"Sucesso!"</strong> para todas as operações válidas. As instruções ```ADD x9, x8, x8``` e ```DIV x10, x1, x0``` deverão mostrar mensagens de ```ERRO: Overflow...``` e ```ERRO: Divisão por zero```, respectivamente. O programa não deve parar.
- <strong>Registradores:</strong> Ao clicar em "Show Registers", verifique se:
  - ```x4```=110, ```x5```=90, ```x6```=70, ```x7```=10.
  - ```x9``` e ```x10``` permanecem ```0```, pois suas operações falharam.
  - Na coluna "Valor FP", ```x13``` deve ser ```4.0000``` e ```x14``` deve ser ```3.7500```.

## <strong>Experimento 2: Validação da Parte 3 (Pipeline Básico)</strong>

<strong>Objetivo:</strong> Demonstrar o funcionamento do pipeline com 4 estágios, mostrando a tabela de evolução e comparando o número de ciclos com o modelo de ciclo único.

<strong>Código Assembly:</strong>


In [255]:
display(create_copy_button(codigo_exp2, button_id=2))

Configurações do Simulador:
- <strong>Simular:</strong> ```Pipeline (Completo)```
- <strong>Hazards:</strong> ```Sem tratamento de hazards``` (para evitar interferência)
- <strong>Memória:</strong> ```Sem Cache```

Análise dos Resultados Esperados:
- <strong>Tabela do Pipeline:</strong> A tabela exibida na área de output deve mostrar o fluxo diagonal das instruções, preenchendo o pipeline nos primeiros 4 ciclos, executando em capacidade máxima, e depois esvaziando.

- <strong>Contagem de Ciclos:</strong> O log final deve mostrar um ```Total de Ciclos (Pipeline)``` de 8 (ou 9, dependendo da condição de parada do loop). Isso valida a fórmula ```Nº de Instruções + Nº de Estágios - 1 = 5 + 4 - 1 = 8```. O ```Total de Ciclos (Ciclo Único seria)``` será 5.

## <strong>Experimento 3: Validação da Parte 4 e Ponto Extra (Análise de Hazards)</strong>

<strong>Objetivo:</strong> Demonstrar a detecção de data hazards, comparar a eficácia das soluções com stalls e forwarding, e responder à pergunta: <strong>*"Qual a diferença entre a solução com stalls e com forwarding?"*</strong>

<strong>Código Assembly:</strong>


In [256]:
display(create_copy_button(codigo_exp3, button_id=3))

Configurações do Simulador (Execute 3 vezes):
- <strong>Run 1 (Stall):</strong> Selecione ```Pipeline```, ```Stall em hazards```, ```Sem Cache```.
- <strong>Run 2 (Forwarding):</strong> Mude apenas para ```Forwarding```.
- <strong>Run 3 (Sem Tratamento):</strong> Mude apenas para ```Sem tratamento de hazards```.

Análise dos Resultados Esperados:

- <strong>Run 1 (Stall):</strong> O log de "Detalhes de Hazards" mostrará <strong>2 hazards</strong> e a ação de <strong>STALL</strong> para ambos. O ```Número de Stalls``` será > 0 e o ```Total de Ciclos será alto```. Os resultados nos registradores (```x4``` e ```x7```) estarão <strong>corretos</strong>.

- <strong>Run 2 (Forwarding):</strong> O log mostrará uma ação de <strong>FORWARDING</strong> para o primeiro hazard e um <strong>STALL OBRIGATÓRIO</strong> para o hazard de Load-Use. O ```Número de Stalls``` será menor que no Run 1, e o ```Total de Ciclos``` também será menor. Os resultados estarão <strong>corretos</strong>.

- <strong>Run 3 (Sem Tratamento):</strong> O log mostrará a <strong>detecção dos 2 hazards<strong>, mas a ação será </strong>"Nenhuma"</strong>. O ```Número de Stalls``` será 0, e o ```Total de Ciclos``` será o menor de todos. No entanto, os valores finais de ```x4``` e ```x7``` estarão <strong>incorretos</strong>, pois as operações usaram dados antigos.

- <strong>Resposta ao Ponto Extra:</strong> A diferença é que a solução com *stalls* para o pipeline, perde ciclos ao colocar muitos stalls, enquanto o *forwarding* resolve a dependência sem parar o pipeline na maioria dos casos, resultando em menos stalls e um menor tempo de execução total, mantendo a correção do programa.

## <strong>Experimento 4: Validação da Parte 5 e Ponto Extra (Cache)</strong>

<strong>Objetivo:</strong> Demonstrar o funcionamento da cache, calcular hits/misses, e responder às perguntas: <strong>*"Como a taxa de acerto da cache afetou o tempo de execução?"*</strong> e <strong>*"Comparando diferentes políticas de substituição de cache (LRU vs Random)."*</strong>

<strong>Código Assembly:</strong>


In [257]:
display(create_copy_button(codigo_exp4, button_id=4))

Configurações do Simulador (Execute 3 vezes):
- <strong>Run 1 (LRU com Penalidade):</strong> Selecione ```Pipeline```, ```Forwarding```, ```Com Cache```, ```LRU```, ```Penalidade Miss: 10```.

- <strong>Run 2 (Random com Penalidade):</strong> Mude apenas para ```Random```.

- <strong>Run 3 (LRU com Alta Penalidade):</strong> Mude para ```LRU``` e ```Penalidade Miss: 50```.

Análise dos Resultados Esperados:

- <strong>Run 1 (LRU):</strong> Anote a taxa de acertos e o ```Total de Ciclos```. A última leitura (```LW x4, 10```) provavelmente será um <strong>miss</strong>, pois o acesso a ```18``` e ```26``` fez com que o bloco ```10``` se tornasse o menos recentemente usado.

- <strong>Run 2 (Random):</strong> A taxa de acertos pode ser <strong>diferente</strong> da de LRU. A última leitura (```LW x4, 10```) pode ser um <strong>hit</strong> se a política ```Random``` tiver sorte e expulsar o bloco ```18``` em vez do ```10```. Isso demonstra a natureza não-determinística da política Random.

- <strong>Run 3 (Alta Penalidade):</strong> A taxa de acertos será a mesma do Run 1, mas o ```Total de Ciclos``` será <strong>significativamente maior</strong>.

- <strong>Respostas aos Pontos Extras:</strong>

  - <strong>Impacto da Taxa de Acerto:</strong> Comparando Run 1 e Run 3, você mostra que, embora a taxa de acerto não mude, o custo dos misses (a penalidade) aumenta drasticamente o tempo de execução, provando que uma alta taxa de acerto é crucial para o desempenho.

  - <strong>Políticas de Substituição:</strong> Comparando Run 1 e Run 2, você mostra que a política de substituição pode levar a diferentes taxas de acerto para o mesmo padrão de acesso.



## <strong>Experimento 5: Validação de Pontos Extras (Control Hazards e Speedup Final)</strong>

<strong>Objetivo:</strong> Demonstrar a simulação de desvios e control hazards, e medir o speedup final do sistema otimizado em relação ao ciclo único.

<strong>Código Assembly:</strong>


In [258]:
display(create_copy_button(codigo_exp5, button_id=5))

Configurações do Simulador (Execute 2 vezes):

- <strong>Run 1 (Baseline):</strong> Selecione ```Ciclo Único```.

- <strong>Run 2 (Otimizado):</strong> Selecione ```Pipeline```, ```Forwarding```, ```Com Cache```, ```LRU```, ```Penalidade Miss: 10```.

Análise dos Resultados Esperados:

- <strong>Run 2 (Otimizado):</strong> O log de "Detalhes de Hazards" mostrará múltiplos ```CONTROL HAZARD``` ```(BEQ)``` e ```(J)```, indicando que os desvios foram detectados e o pipeline foi esvaziado (flushed), o que adiciona ciclos de penalidade.

- <strong>Registradores e Memória:</strong> Em ambos os runs, o resultado final deve ser <strong>correto</strong>. O registrador ```x2``` deve terminar com o valor ```50```, e o endereço de memória ```50``` também conterá ```50```.

- <strong>Resposta ao Ponto Extra (Speedup):</strong> Pegue o ```Total de Ciclos``` do Run 1 (Ciclo Único) e do Run 2 (Pipeline). Calcule o <strong>Speedup = Ciclos_Único / Ciclos_Pipeline</strong>. O log do Run 2 já fará isso para você. Este valor quantifica o ganho de desempenho total de todas as otimizações (pipeline, forwarding, cache) em relação ao design mais simples, mesmo considerando as penalidades de hazards de controle e cache misses.